In [1]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import os
import re

In [24]:
company_data = pd.read_csv('company_email.csv')
# Remove Chinese characters across all string columns
company_data = company_data.applymap(lambda x: re.sub(r'[\u4e00-\u9fff]+', '', x) if isinstance(x, str) else x)

# Clean company names: remove empty parentheses, remove any parenthetical content, trim whitespace
plain_company_name = []
for name in company_data['Company'].fillna('').astype(str).tolist():
    # remove literal empty parentheses '()'
    cleaned = re.sub(r"\(\)", '', name)
    # remove any parentheses and their content (e.g. 'Company (HK)' -> 'Company')
    cleaned = re.sub(r"\s*\(.*?\)", '', cleaned).strip()
    plain_company_name.append(cleaned)

# attach cleaned names back to the dataframe for later use
company_data['Company'] = plain_company_name

# show result
plain_company_name

/var/folders/_q/0zd4szy510g43b2ptl354v340000gn/T/ipykernel_59643/499939971.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  company_data = company_data.applymap(lambda x: re.sub(r'[\u4e00-\u9fff]+', '', x) if isinstance(x, str) else x)


['The Matcher Limited',
 'Beijing Science and Technology Co., Three Fast',
 'Shenzhen Xunlei Network Technology Co., Ltd',
 'Shenzhen Xunlei Network Technology Co., Ltd',
 'OKBL Technology Company Limited',
 'OKBL Technology Company Limited',
 'Linjer Limited',
 'Shenzhen Xunlei Network Technology Co., Ltd',
 'Shenzhen Xunlei Network Technology Co., Ltd',
 'Shenzhen Xunlei Network Technology Co., Ltd',
 'Shenzhen Xunlei Network Technology Co., Ltd',
 'Shenzhen Xunlei Network Technology Co., Ltd',
 'Shenzhen Xunlei Network Technology Co., Ltd',
 'Hong Kong Technology Venture Company Limited',
 'Baidu Online Network Technology Co., Ltd.',
 'Baidu Online Network Technology Co., Ltd.',
 'Baidu Online Network Technology Co., Ltd.',
 'Baidu Online Network Technology Co., Ltd.',
 'Baidu Online Network Technology Co., Ltd.',
 'YesStyle.com',
 'Baidu Online Network Technology Co., Ltd.',
 'Baidu Online Network Technology Co., Ltd.',
 'Baidu Online Network Technology Co., Ltd.',
 'DiDi Global In

In [25]:
company_data.dropna(axis=0,inplace=True)

In [26]:
company_data.to_csv('company_email_plain.csv',index=False,encoding="utf-8")

In [27]:
#optional: Remove duplicated company and email
company_data = pd.read_csv('company_email_plain.csv')
company_data = company_data.drop_duplicates(subset=["Company"], keep="first")

In [29]:
company_data.head()

,Company,Role,Email
0,The Matcher Limited,R&D Software Engineer,recruitment@the-matcher.com
1,"Shenzhen Xunlei Network Technology Co., Ltd",【（web3）】,campus@xunlei.com
10,Wave Commerce,Business Associate (Business Development & Cli...,jobs+ba@wavecommerce.hk
11,Delivery Hero Food Hong Kong Limited,Sales Executive,hk_recruit@foodpanda.hk
12,Pattern Holdings Company Limited,Management Trainee,grace.wong@patternasia.com


In [23]:
company_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 0 to 321
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Company  97 non-null     object
 1   Role     97 non-null     object
 2   Email    97 non-null     object
dtypes: object(3)
memory usage: 3.0+ KB


In [17]:
# Gmail setup
gmail_user = "jasperlung0418@gmail.com"
app_password = "kpab memw xwsu uvny"
resume_path = 'Lung Chi Hang Jasper CV.pdf'

for role, company, recipient in zip(company_data["Roles"], company_data["Company"], company_data["Email"]):
    subject = f'Application for {role} Position'
    body = f"""Dear Hiring Manager,

I hope this message finds you well. I am writing to express my interest for {role} in {company}.

Attached are my Resume for your review.

Thank you for your consideration. I look forward to hearing from you soon.

Best Regards,
Jasper Lung
"""
    
    msg = MIMEMultipart()
    msg["From"] = gmail_user
    msg["To"] = recipient
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

#Attach Resume
    if os.path.exists(resume_path):
        with open(resume_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(resume_path))
        part["Content-Disposition"] = f'attachment; filename="{os.path.basename(resume_path)}"'
        msg.attach(part)
    else:
        print("Resume not found, skipping attachment.")
    
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(gmail_user, app_password)
        server.sendmail(gmail_user, recipient, msg.as_string())




In [ ]:
# Test
gmail_user = "YOUR GMAIL"
app_password = "YOUR APP PWD"
resume_path = 'YOUR RESUME'

subject = f'Application for {company_data.Roles.tolist()[0]} Position'
body = f"""Dear Hiring Manager,

I am writing to express my enthusiasm for the {company_data.Roles.tolist()[0]} position at {company_data.Company.tolist()[0]}. 
I am eager to contribute my skills and motivation to your team, and I am confident 
that my background and dedication make me a strong candidate for this role.

Please find my resume attached for your review. 
I would welcome the opportunity to discuss how my qualifications align with your needs.

Best Regards,
Kay
"""

msg = MIMEMultipart()
msg['From'] = gmail_user
msg['To'] = company_data.Email.tolist()[0]
msg['Subject'] = subject
msg.attach(MIMEText(body, "plain"))

if os.path.exists(resume_path):
    with open(resume_path, "rb") as f:
        part = MIMEApplication(f.read(), Name=os.path.basename(resume_path))
    part["Content-Disposition"] = f'attachment; filename="{os.path.basename(resume_path)}"'
    msg.attach(part)
else:
    print("Resume not found, skipping attachment.")

with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(gmail_user, app_password)
    server.sendmail(gmail_user, company_data.Email.tolist()[0], msg.as_string())


In [13]:
resume_path = 'Lung Chi Hang Jasper CV.pdf'
if os.path.exists(resume_path):
    print('Done')
else:
    print('No')

Done
